# Conda Environment Setup

In [ ]:
'''
name: birth
channels:
  - conda-forge
  - defaults
dependencies:
  - python
  - pandas
  - matplotlib
  - seaborn
  - scikit-learn
  - scipy
  - statsmodels
  - pymc3
  - arviz
  - ipywidgets
'''

# Data Preprocessing

This notebook conducts data preprocessing, including data cleaning, feature engineering, and subsampling.

In [ ]:
import pandas as pd

In [ ]:
birth = pd.read_csv('US_births(2018).csv')

In [ ]:
birth.shape

## Data Cleaning

In [ ]:
clean_birth = birth.dropna()

# remove missing values in the response/outcome variable
clean_birth = clean_birth[clean_birth['DBWT'] != 9999] 

# remove missing values in the features
clean_birth = clean_birth[
    (clean_birth['PRECARE'] != 99) & 
    (clean_birth['CIG_0'] != 99) & 
    (clean_birth['BMI'] != 99.9) & 
    (clean_birth['PREVIS'] != 99) & 
    (clean_birth['MRAVE6'] != 9) & 
    (clean_birth['PAY_REC'] != 9) &
    (clean_birth['FRACE6'] != 9) &
    (clean_birth['MEDUC'] != 9) & 
    (clean_birth['FEDUC'] != 9) & 
    (clean_birth['NO_RISKS'] != 9) & 
    (clean_birth['ATTEND'] != 9) &
    (clean_birth['BFACIL'] != 9) &
    (clean_birth['FAGECOMB'] != 99) &
    (clean_birth['RF_CESAR'] != 'U') &
    (clean_birth['LD_INDL'] != 'U') &
    (clean_birth['MBSTATE_REC'] != 3) &
    (clean_birth['M_Ht_In'] != 99) &
    (clean_birth['NO_INFEC'] != 9) &
    (clean_birth['NO_MMORB'] != 9) &
    (clean_birth['PRIORLIVE'] != 99) &
    (clean_birth['PRIORTERM'] != 99) &
    (clean_birth['RDMETH_REC'] != 9) &
    (clean_birth['DLMP_YY'] != 9999) &
    (clean_birth['DLMP_MM'] != 99) &
    (clean_birth['PWgt_R'] != 999) &
    (clean_birth['WTGAIN'] != 99) &
    (clean_birth['ILLB_R'] != 909)
] 

In [ ]:
clean_birth.shape

## Feature engineering

In [ ]:
# estimate pregnancy length
clean_birth['PREG_LEN'] = 12*(2018 - clean_birth['DLMP_YY']) + (clean_birth['DOB_MM'] - clean_birth['DLMP_MM']) 

# recode PRECARE
clean_birth['PRECARE'][(clean_birth['PRECARE'] < 4) & (clean_birth['PRECARE'] > 0)] = 1 
clean_birth['PRECARE'][(clean_birth['PRECARE'] < 7) & (clean_birth['PRECARE'] > 3)] = 2
clean_birth['PRECARE'][(clean_birth['PRECARE'] > 6)] = 3

# compute percentage weight gain
clean_birth['WTGAIN_PER'] = clean_birth['WTGAIN'] / clean_birth['PWgt_R'] 

# binarize CIG_0
clean_birth['CIG'] = clean_birth['CIG_0'] > 0 

# binarize PRIORDEAD
clean_birth['PRIORDEAD'] = clean_birth['PRIORDEAD'] > 0

# binarize PRIORTERM
clean_birth['PRIORTERM'] = clean_birth['PRIORTERM'] > 0

# binarize PRIORLIVE
clean_birth['PRIORLIVE'] = clean_birth['PRIORLIVE'] > 0

# compute first time live birth
clean_birth['FIRST_BIRTH'] = clean_birth['ILLB_R'] == 888

In [ ]:
# drop columns where >99% entries are the same
clean_birth = clean_birth.drop(['DOB_YY', 'IMP_SEX', 'IP_GON', 'MAGE_IMPFLG', 'MAR_IMP', 'MM_AICU', 'MTRAN'], axis=1)

# drop redundant columns due to feature engineering
clean_birth = clean_birth.drop(['WTGAIN', 'PWgt_R', 'DWgt_R', 'DOB_MM', 
                                   'DOB_WK', 'DOB_TT', 'DOB_MM', 'DLMP_YY',
                                   'DLMP_MM', 'PAY', 'MHISPX', 'MRACE15',
                                   'MRACE31', 'MRACEIMP', 'FHISPX', 'FRACE15',
                                   'FRACE31', 'RF_CESARN', 'ILOP_R', 'ILP_R', 'ILLB_R','CIG_0'], axis=1)

In [ ]:
clean_birth.shape

## Subsampling

In [ ]:
n = 10000
sub_clean_birth = clean_birth.sample(n, random_state=102)
sub_clean_birth.to_csv('subsampled_clean_data.csv')

# EDA

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
birth = pd.read_csv('subsampled_clean_data.csv')

In [ ]:
birth.head()

## Quantitative Variables

## Birth Weight – Detail in Grams (DBWT)

In [ ]:
sns.histplot(birth['DBWT'], stat = 'density', kde = True)
plt.title("Birth Weight-Detail in Grams")

In [ ]:
#Mean
np.mean(birth['DBWT'])

In [ ]:
#SD
np.std(birth['DBWT'])

In [ ]:
#min and max
np.min(birth['DBWT']), np.max(birth['DBWT'])

## Mother’s Single Years of Age (MAGER)

In [ ]:
sns.histplot(birth['MAGER'], stat = 'density')
plt.title("Mother's Single Years of Age")

In [ ]:
#Mean
np.mean(birth['MAGER'])

In [ ]:
#SD
np.std(birth['MAGER'])

In [ ]:
#min and max
np.min(birth['MAGER']), np.max(birth['MAGER'])

## Number of Prenatal Visits - PREVIS

In [ ]:
sns.histplot(birth['PREVIS'], stat = 'density')
plt.title("Number of Prenatal Visits")

In [ ]:
#Mean
np.mean(birth['PREVIS'])

In [ ]:
#SD
np.std(birth['PREVIS'])

In [ ]:
#min and max
np.min(birth['PREVIS']), np.max(birth['PREVIS'])

# Categorical Variables

## Smoking Before Pregnancy - CIG

In [ ]:
sns.countplot(x = birth['CIG'])
plt.title("Smoking Before Pregnancy")

In [ ]:
#Number of Smokers
np.sum(birth['CIG'] == 1)

In [ ]:
#Number of Non-smokers
np.sum(birth['CIG'] == 0)

In [ ]:
#Percentage of each category
freq = birth['CIG'].value_counts()
perc = freq/sum(freq)
perc

## First Birth - FIRST_BIRTH

In [ ]:
sns.countplot(x = birth['FIRST_BIRTH'])
plt.title("First Birth Child")

In [ ]:
#Number of First Birth
np.sum(birth['FIRST_BIRTH'] == 1)

In [ ]:
#Number of None First Birth
np.sum(birth['FIRST_BIRTH'] == 0)

In [ ]:
#Percentage of each category
freq = birth['FIRST_BIRTH'].value_counts()
perc = freq/sum(freq)
perc

## Sex of Infant - SEX

In [ ]:
sns.countplot(x = birth['SEX'])
plt.title("Babies' Biological Gender")

In [ ]:
#Number of Males
np.sum(birth['SEX'] == 'M')

In [ ]:
#Number of Females
np.sum(birth['SEX'] == 'F')

In [ ]:
#Percentage of each category
freq = birth['SEX'].value_counts()
perc = freq/sum(freq)
perc

# Causal Inference

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
from scipy.stats import beta, binom
import itertools
from ipywidgets import interact, interactive
import statsmodels.api as sm

import sklearn
from sklearn.linear_model import LogisticRegression as LR

In [ ]:
#load dataset
birth = pd.read_csv('subsampled_clean_data.csv')
birth

## Causal Inference - Randomized Experiments

In [ ]:
observed_T = np.mean(birth[birth["CIG"] == True].DBWT) - np.mean(birth[birth["CIG"] == False].DBWT)
observed_T

In [ ]:
birth['CIG'].sum()

In [ ]:
rng = np.random.default_rng(102)
R = 50000 # repetition times
Ts = np.zeros(R)
shuffled_birth = birth.copy()

for i in range(R):
    shuffled_birth['shuffled_CIG'] = rng.choice(birth['CIG'], size=birth.shape[0], replace=False)
    Ts[i] = np.mean(shuffled_birth[shuffled_birth["shuffled_CIG"] == True].DBWT) - np.mean(shuffled_birth[shuffled_birth["shuffled_CIG"] == False].DBWT)

p_val = np.sum(np.abs(Ts) >= np.abs(observed_T)) / R
print(f'The p-value is {p_val}')

fig, ax = plt.subplots()
ax.hist(Ts, density=True, label='Null Distribution')
ax.axvline(observed_T, color='r', label='Observed')
ax.set_xlabel('Test Statistics')
fig.legend()
fig.show()

## Causal Inference - Observational Studies

## Outcome Regression

In [ ]:
#change categorical data to dummy variable
birth = pd.get_dummies(birth, columns=['CIG'], drop_first=True)
birth 

In [ ]:
def fit_OLS_model(df, target_variable, explanatory_variables, intercept = False):
    target = df[target_variable]
    inputs = df[explanatory_variables]
    if intercept:
        inputs = sm.add_constant(inputs)
    
    fitted_model = sm.OLS(target, inputs).fit()
    return(fitted_model)

def mean_squared_error(true_vals, predicted_vals):
    return np.mean((true_vals - predicted_vals) ** 2)

In [ ]:
full_linear_model = fit_OLS_model(birth, 'DBWT', ['CIG_True', 'BMI', 'PRECARE'], True)
print(full_linear_model.summary())

In [ ]:
sns.histplot(data=birth, x='DBWT', hue='CIG_True', stat='density')
plt.title("Distributions of Babies' Weights under Different Cigarettes Usage")

In [ ]:
birth.groupby('CIG_True')['DBWT'].mean()

In [ ]:
sns.histplot(data=birth, x='DBWT', hue='PRECARE', stat='density')
plt.title("Distributions of Babies' Weights under Different Prenatal Care Levels")

In [ ]:
birth.groupby('PRECARE')['DBWT'].mean()

### Inverse Propensity Score

In [ ]:
Z = birth.CIG_True.values
Y = birth.DBWT.values
X = birth[['BMI', 'PRECARE']]

lr = LR(max_iter=200, random_state=0)
lr.fit(X, Z)

In [ ]:
birth['pscore'] = lr.predict_proba(X)[:,1]
birth

In [ ]:
n = len(birth)
ipw = np.sum(birth[birth["CIG_True"] == 1].DBWT / birth[birth["CIG_True"] == 1].pscore)/n - np.sum(birth[birth["CIG_True"] == 0].DBWT / (1 - birth[birth["CIG_True"] == 0].pscore))/n
ipw

In [ ]:
plt.hist(birth[birth['CIG_True'] == 1]['pscore']);
plt.title("Propensity score of people receiving the treatment");

In [ ]:
plt.hist(birth[birth['CIG_True'] == 0]['pscore']);
plt.title("Propensity score of people not receiving the treatment");

# Prediction

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor

In [ ]:
birth = pd.read_csv('subsampled_clean_data.csv')

In [ ]:
birth.head()

# Nonparametric Method-Decision Tree

### Baby Weights (y)

In [ ]:
y = np.array(birth['DBWT'])
y

### Valid features to construct the tree (X)

In [ ]:
# Drop irrelevant features and y
X_drop = birth.drop(['DBWT', 'DMAR'], axis = 1)
X_drop.head()

In [ ]:
# Get all the categorical data
cat_cols = X_drop.select_dtypes(exclude=["number"]).columns
cat_cols

In [ ]:
for c in cat_cols:
    encoded = pd.get_dummies(X_drop[c], prefix=c)
    X_drop = pd.concat([X_drop, encoded], axis='columns')

In [ ]:
X_encoded = X_drop.drop(cat_cols, axis = 1)

In [ ]:
X_encoded.head()

### Fit

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.01, random_state=0)

In [ ]:
depths = [i for i in range(1, 10)]
train_scores = np.ones(len(depths))
test_scores = np.ones(len(depths))

#### Cross validation

In [ ]:
for idx in range(len(depths)):
    clf = tree.DecisionTreeRegressor(max_depth = depths[idx])
    clf.fit(X_train, y_train)
    train_scores[idx] = clf.score(X_train, y_train)
    test_scores[idx] = clf.score(X_test, y_test)
    print("Max depths" ,depths[idx] , "will have train score" , train_scores[idx] , "and test score" , test_scores[idx])

According to the cross validation, max_depth = 4 looks promising, so that's what we will use in the prediction phase

In [ ]:
clf = tree.DecisionTreeRegressor(max_depth = 4)
clf.fit(X_train, y_train)

### Summarize and interpret 

#### Visualize tree

In [ ]:
plt.figure(figsize=(10,10))
tree.plot_tree(clf, fontsize=9)
plt.show()

In [ ]:
clf.feature_names_in_[19], clf.feature_names_in_[10], clf.feature_names_in_[16]

#### Ploting the true labels and predicted labels

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
x_ax = range(len(X_test))
plt.plot(x_ax, y_test, label = 'true label', color = 'k', linestyle = '-')
plt.plot(x_ax, clf.predict(X_test), label = 'predicted', color = 'k', linestyle = '--')
plt.ylabel("Baby's birth weights")
plt.xlabel("Testing sample data")
plt.legend()
plt.show()

# Nonparametric Method-Random Forest

In [ ]:
clf1 = RandomForestRegressor(n_estimators= 200,max_depth=15)

In [ ]:
clf1.fit(X_train, y_train)

In [ ]:
clf1.score(X_train, y_train)

In [ ]:
clf1.score(X_test, y_test)

#### Ploting the true labels and predicted labels

In [ ]:
plt.rcParams['figure.figsize'] = (10, 6)
x_ax = range(len(X_test))
plt.plot(x_ax, y_test, label = 'true label', color = 'k', linestyle = '-')
plt.plot(x_ax, clf1.predict(X_test), label = 'predicted', color = 'k', linestyle = '--')
plt.ylabel("Baby's birth weights")
plt.xlabel("Testing sample data")
plt.legend()
plt.show()

# GLM

In [ ]:
#import the pymc3 package
import pymc3 as pm
from pymc3 import glm
import statsmodels.api as sm
import arviz as az

## Choice of Model - Linear Regression

## Frequentist Regression

In [ ]:
freq_model = sm.GLM(birth.DBWT, exog = sm.add_constant(birth[['MAGER','PREVIS']]), 
                  family=sm.families.Gaussian())
freq_res = freq_model.fit()
print(freq_res.summary())

### Model Checking

In [ ]:
from statsmodels.graphics.api import abline_plot

In [ ]:
nobs = freq_res.nobs
y = np.array(birth['DBWT'])
yhat = freq_res.mu

In [ ]:
fig, ax = plt.subplots()
ax.scatter(yhat, y)
line_fit = sm.OLS(y, sm.add_constant(yhat, prepend=True)).fit()
abline_plot(model_results=line_fit, ax=ax)


ax.set_title('Model Fit Plot')
ax.set_ylabel('Observed values')
ax.set_xlabel('Fitted values');

In [ ]:
from statsmodels import graphics
resid = freq_res.resid_deviance.copy()
graphics.gofplots.qqplot(resid, line='r')

## Bayesian Regression

In [ ]:
dbwt = np.array(birth['DBWT'])
mager = np.array(birth['MAGER'])
previs = np.array(birth['PREVIS'])

In [ ]:
with pm.Model() as bayes_model:
    #define the priors
    sigma = pm.Exponential('sigma', lam=0.01)
    intercept = pm.Normal("Intercept", 3015, sigma=30)
    beta_1 = pm.Normal("beta_1", 2, sigma=3)
    beta_2 = pm.Normal("beta_2", 18, sigma=3)
    
    #likelihood
    likelihood = pm.Normal("y", mu = intercept + beta_1*mager + beta_2*previs, sigma = sigma, observed = dbwt)
    
    #inference
    trace = pm.sample(1000, cores = 2, target_accept = 0.95, return_inferencedata=True)

In [ ]:
az.plot_trace(trace, figsize=(20, 15))

In [ ]:
np.mean(trace.posterior["Intercept"].values)

In [ ]:
np.mean(trace.posterior["beta_1"].values)

In [ ]:
np.mean(trace.posterior["beta_2"].values)

### Model Checking

In [ ]:
with bayes_model:
    ppc = pm.sample_posterior_predictive(
        trace, var_names=["beta_1", "beta_2", "Intercept", "y"]
    )

In [ ]:
ppc

In [ ]:
for i in np.arange(100):
    sns.histplot(ppc['y'][i])
plt.title("PPC of Birth Weights")

In [ ]:
#Actual birthweight histogram
sns.histplot(birth['DBWT'])
plt.title("Actual distribution of Birth Weights")

### Uncertainty Quantification

In [ ]:
az.plot_posterior(trace, ['Intercept', 'beta_1', 'beta_2'], round_to = 3)